# JOB-SHOP PROBLEM WITH SIMULATED ANNEALING

In [30]:
import pandas as pd
import random
import math

In [31]:
def read_data(file):
    df = pd.read_excel(file, header=None)
    matrix = []

    # Iterate over rows to process job data
    for job in df.itertuples(index=False):
        # Convert the row into pairs of (machine, time)
        row_data = list(job)
        job_data = {}
        for i in range(0, len(row_data), 2):
            machine = int(row_data[i])
            time = int(row_data[i+1])
            job_data[machine] = time
        matrix.append(job_data)

    result_df = pd.DataFrame(matrix).fillna(0).astype(int)
    return result_df

In [32]:
data = read_data("JSSP-problems/basic-problem.xlsx")
data

,1,0,2
0,2,3,2
1,4,2,1
2,4,0,3


In [33]:
def fitness(chromosome, data):
    job_end_time = [0] * data.shape[0]  # End time of each job
    machine_end_time = [0] * data.shape[1]  # End time of each machine
    
    for task in chromosome:
        job, machine = map(int, task)
        start_time = max(job_end_time[job], machine_end_time[machine]) # Start time of the task
        job_end_time[job] = start_time + data[machine][job] # Update end time of job
        machine_end_time[machine] = start_time + data[machine][job] # Update end time of machine
    
    return max(machine_end_time)  # Makespan

In [34]:
def initial_configuration(data):
    jobs = list(range(data.shape[0]))
    machines = list(range(data.shape[1]))
    chromosome = []
    
    for job in jobs:
        for machine in machines:
            chromosome.append((job, machine))
    
    random.shuffle(chromosome)
    return chromosome

In [35]:
def get_neighbours(chromosome):
    neighbor = chromosome[:]
    idx1, idx2 = random.sample(range(len(chromosome)), 2)
    neighbor[idx1], neighbor[idx2] = neighbor[idx2], neighbor[idx1]
    return neighbor

[(2, 0), (2, 1), (1, 1), (1, 0), (0, 1), (1, 2), (0, 0), (2, 2), (0, 2)]

In [36]:
def simulated_annealing(data, params):
    initial_solution = initial_configuration(data)
    current_solution = initial_solution
    current_fitness = fitness(current_solution, data)
    best_solution = current_solution
    best_fitness = current_fitness

    initial_temperature = params["initial_temperature"]
    cooling_rate = params["cooling_rate"]
    max_iterations = params["max_iterations"]

    temperature = initial_temperature
    fitness_progression = [current_fitness]

    for iteration in range(max_iterations):
        # Generate a neighbor solution
        neighbor_solution = get_neighbours(current_solution)
        neighbor_fitness = fitness(neighbor_solution, data)

        # Calculate acceptance probability
        delta_fitness = neighbor_fitness - current_fitness
        acceptance_probability = math.exp(-delta_fitness / temperature) if delta_fitness > 0 else 1.0

        # Accept or reject the neighbor
        if random.random() < acceptance_probability:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness

            # Update the best solution found
            if current_fitness < best_fitness:
                best_solution = current_solution
                best_fitness = current_fitness

        # Cool down
        temperature *= cooling_rate

        fitness_progression.append(best_fitness)

        # Stop if temperature is too low
        if temperature < 1e-8:
            break

    return best_solution, best_fitness, fitness_progression

In [37]:
params = {
    "initial_temperature": 1000, 
    "cooling_rate": 0.99,         
    "max_iterations": 10000    
}

In [ ]:
best_solution, best_fitness, fitness_progression = simulated_annealing(data, params)

Best solution: [(2, 1), (0, 0), (2, 0), (1, 2), (1, 0), (0, 1), (2, 2), (1, 1), (0, 2)]
